Building a Chatbot

This notebook shows how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")


In [19]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017E5CC92380>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017E5CC925C0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [20]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Mounica. I'm currently learning chatbot building using LangChain")])


AIMessage(content='Hello Mounica, nice to meet you. LangChain is a great library for building chatbots, it allows for a lot of flexibility and customization. What specific aspects of chatbot building are you interested in learning or implementing with LangChain? Are you working on a project already or just exploring the library?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 53, 'total_tokens': 116, 'completion_time': 0.100512984, 'prompt_time': 0.003738742, 'queue_time': 0.049893428, 'total_time': 0.104251726}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e7440ff1-9e6c-4f66-bbfe-749bb6a35a8c-0', usage_metadata={'input_tokens': 53, 'output_tokens': 63, 'total_tokens': 116})

In [21]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hi, I am Mounica. I'm currently learning chatbot building using LangChain"), 
AIMessage(content="Hello Mounica, nice to meet you. LangChain is a powerful framework for building chatbots and conversational AI systems. It allows for seamless integration with various language models and enables developers to create more complex and context-aware conversational flows.\n\nWhat specific aspects of chatbot building with LangChain are you currently exploring or would like to explore? Are you looking to implement a specific feature or functionality?"),
(HumanMessage(content="What's my name and what am i learning"))])



AIMessage(content="Your name is Mounica, and you're learning chatbot building using the LangChain framework.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_time': 0.027406426, 'prompt_time': 0.009794058, 'queue_time': 0.055551662, 'total_time': 0.037200484}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--ee4b60ae-db7c-4137-afe5-dcd8f40dd7e2-0', usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})

Message History

* We use message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of model, and store them in some datastore.
* Future interactions will then load those messages and pass them into the chain as part of the input.

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store=dict()

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [23]:
config={"configurable": {"session_id":"user1"}}

In [24]:
with_message_history.invoke([HumanMessage(content="Hi, I am Mounica. I'm currently learning chatbot building using LangChain"), 
AIMessage(content="Hello Mounica, nice to meet you. LangChain is a powerful framework for building chatbots and conversational AI systems. It allows for seamless integration with various language models and enables developers to create more complex and context-aware conversational flows.\n\nWhat specific aspects of chatbot building with LangChain are you currently exploring or would like to explore? Are you looking to implement a specific feature or functionality?")], config=config)

AIMessage(content=" I'm here to help and provide guidance as needed.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 133, 'total_tokens': 145, 'completion_time': 0.01659854, 'prompt_time': 0.009703259, 'queue_time': 0.051164041, 'total_time': 0.026301799}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--48af8c4d-307a-4caf-a904-33aea4bfb31d-0', usage_metadata={'input_tokens': 133, 'output_tokens': 12, 'total_tokens': 145})

In [25]:
with_message_history.invoke(HumanMessage(content="waht is my name?"), config=config)

AIMessage(content='Your name is Mounica.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 165, 'total_tokens': 173, 'completion_time': 0.004171802, 'prompt_time': 0.010657028, 'queue_time': 0.055175062, 'total_time': 0.01482883}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--7276d55c-a429-4c6e-b712-15d312b9b124-0', usage_metadata={'input_tokens': 165, 'output_tokens': 8, 'total_tokens': 173})

In [26]:
config2={"configurable":{"session_id":"user2"}}
with_message_history.invoke("waht is my name?", config=config2)

AIMessage(content="I don't have any information about your name as this is the start of our conversation. I'm happy to chat with you, though! If you'd like to share your name, I'd be happy to learn it.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 41, 'total_tokens': 87, 'completion_time': 0.064600083, 'prompt_time': 0.003154215, 'queue_time': 0.051373785, 'total_time': 0.067754298}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--fedf49c2-96e0-48c7-9c1a-9dcecb9dc66a-0', usage_metadata={'input_tokens': 41, 'output_tokens': 46, 'total_tokens': 87})

since session_id is different and different config is passed now, it's not able to recognize name of user.

Working with prompt template and message chat history

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in selected language: {language}"), 
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [28]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages", )

In [29]:
config3 = {"configurable": {"session_id": "user3"}}

In [30]:
response=with_message_history.invoke({"language":"hindi", "messages":"Hi, My name is Mounica"}, config=config3)
response

AIMessage(content='नमस्ते मुनिका! मैं आपकी सहायता करने के लिए तैयार हूँ। क्या मैं आपकी किसी भी समस्या या प्रश्न का समाधान कर सकता हूँ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 64, 'total_tokens': 125, 'completion_time': 0.068396814, 'prompt_time': 0.004152551, 'queue_time': 0.054156769, 'total_time': 0.072549365}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e2e0e441-e6dd-443a-90c0-56c2e540e984-0', usage_metadata={'input_tokens': 64, 'output_tokens': 61, 'total_tokens': 125})

In [31]:
response= with_message_history.invoke({"language":"telugu", "messages":"what is my name?"}, config=config3)
response


AIMessage(content='నీవు ఎవరు? నీ పేరు మౌనికా.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 140, 'total_tokens': 183, 'completion_time': 0.081903735, 'prompt_time': 0.008378472, 'queue_time': 0.054399767, 'total_time': 0.090282207}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1bdd94b5-75a2-4f94-8559-d0854baac240-0', usage_metadata={'input_tokens': 140, 'output_tokens': 43, 'total_tokens': 183})

Managing the conversation history

In [47]:
from langchain_core.messages import HumanMessage, trim_messages, SystemMessage, AIMessage

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    allow_partial=True,
    start_on="human",
    include_system=True

)

trimmer

RunnableLambda(...)

In [48]:
messages=[
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="Hi, Im John here"),
    AIMessage(content="Hi John, Nice to meet you"),
    HumanMessage(content="I like Mangoes"),
    AIMessage(content="Great"),
    HumanMessage(content="what is 2 multiple 2"),
    AIMessage(content="it is 4"),
    HumanMessage(content="ok, Thanks"),
    AIMessage(content="No problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes of course!")
    
]

messages

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, Im John here', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi John, Nice to meet you', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Mangoes', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Great', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2 multiple 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='it is 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ok, Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes of course!', additional_kwargs={}, response_metadata={})]

In [49]:
trimmer.invoke(messages)


[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Mangoes', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Great', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2 multiple 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='it is 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ok, Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes of course!', additional_kwargs={}, response_metadata={})]

Using trimmer along with chain

In [50]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
| prompt
| model)

In [51]:
chain.invoke({"messages":[HumanMessage(content="what is maths question which I was speaking previously")], "language":"english"})

AIMessage(content="Unfortunately, I'm a large language model, I don't have the ability to retain information or recall previous conversations. Each time you interact with me, it's a new conversation.\n\nHowever, I can suggest that if you'd like to discuss a math question, you can share it with me now and I'll do my best to help you solve it. What's the math question you'd like to ask?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 65, 'total_tokens': 148, 'completion_time': 0.103026067, 'prompt_time': 0.004485213, 'queue_time': 0.050179847, 'total_time': 0.10751128}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--b4f3d92e-b2b9-4c5d-adf4-3126c907a4e6-0', usage_metadata={'input_tokens': 65, 'output_tokens': 83, 'total_tokens': 148})